# Edit distance

Zadanie dotyczy wykorzystania odległości edycyjnej.

1. Zaimplementuj algorytm obliczania odległości edycyjnej w taki sposób, aby możliwe było określenie przynajmniej jednej sekwencji edycji (dodanie, usunięcie, zmiana znaku), która pozwala w minimalnej liczbie kroków, przekształcić jeden łańcuch w drugi.
2. Na podstawie poprzedniego punktu zaimplementuj prostą wizualizację działania algorytmu, poprzez wskazanie kolejnych wersji pierwszego łańcucha, w których dokonywana jest określona zmiana. "Wizualizacja" może działać w trybie tekstowym. Np. zmiana łańcuch "los" w "kloc" może być zrealizowana następująco:
        *k*los (dodanie litery k)
        klo*c* (zamiana s->c)
3. Przedstaw wynik działania algorytmu z p. 2 dla następujących par łańcuchów:
        los - kloc
        Łódź - Lodz
        kwintesencja - quintessence
        ATGAATCTTACCGCCTCG - ATGAGGCTCTGGCCCCTG
       
4. Zaimplementuj algorytm obliczania najdłuższego wspólnego podciągu dla pary ciągów elementów.
5. Korzystając z gotowego tokenizera (np. spaCy - https://spacy.io/api/tokenizer) dokonaj podziału załączonego tekstu na tokeny.
6. Stwórz 2 wersje załączonego tekstu, w których usunięto 3% losowych tokenów.
7. Oblicz długość najdłuższego podciągu wspólnych tokenów dla tych tekstów.
8. Korzystając z algorytmu z punktu 4 skonstruuj narzędzie, o działaniu podobnym do narzędzia diff, tzn. wskazującego w dwóch plikach linie, które się różnią. Na wyjściu narzędzia powinny znaleźć się elementy, które nie należą do najdłuższego wspólnego podciągu. Należy wskazać skąd dana linia pochodzi (< > - pierwszy/drugi plik) oraz numer linii w danym pliku.
9. Przedstaw wynik działania narzędzia na tekstach z punktu 6. Zwróć uwagę na dodanie znaków przejścia do nowej linii, które są usuwane w trakcie tokenizacji.

## Zad 1. 

In [1]:
def delta1(a,b):
    if a == b:
        return 0
    else: return 1

In [2]:
def get_edit_table(str1, str2, delta = delta1):
    m = len(str1)
    n = len(str2)
    
    edit_table = [[0] * (n + 1) for _ in range(m + 1)]
    for i in range(m + 1):
        for j in range(n + 1):
            
            # insert all letters
            if i == 0:
                edit_table[i][j] = j
                
            # remove all letters
            elif j == 0:
                edit_table[i][j] = i
                
            else:
                d = delta(str1[i-1],str2[j-1])
                edit_table[i][j] = min(edit_table[i][j-1] + 1,   # insert
                                       edit_table[i-1][j] + 1,   # remove
                                       edit_table[i-1][j-1] + d) # replace or do nothing
                                       
    return edit_table

In [3]:
get_edit_table("abcd","abcdab")

[[0, 1, 2, 3, 4, 5, 6],
 [1, 0, 1, 2, 3, 4, 5],
 [2, 1, 0, 1, 2, 3, 4],
 [3, 2, 1, 0, 1, 2, 3],
 [4, 3, 2, 1, 0, 1, 2]]

In [4]:
def edit_distance(str1, str2, delta = delta1):
    return get_edit_table(str1, str2, delta)[-1][-1]

In [5]:
def get_path(a,b,edit_table,delta = delta1):
    seq = []
    pos = []  # position in b (zad 9.) 
    x = len(a)
    y = len(b)
    
    while x > 0 or y > 0:
        current = edit_table[x][y]
        
        if edit_table[x-1][y] + 1 == current:
            seq.append('REMOVE')
            x -= 1
        elif edit_table[x][y-1] + 1 == current:
            seq.append('INSERT')
            y -= 1
        elif edit_table[x-1][y-1] == current:
            seq.append('NOTHING')
            x -= 1
            y -= 1
        else:
            seq.append('REPLACE')
            x -= 1
            y -= 1
        pos.append(y)

    pos.reverse()         
    seq.reverse()
    return seq,pos

## Zad 2.

In [6]:
def show_operations(a,b,path):
    i = j = 0
    current = 1
    print("\n-------- OPERATIONS ------------------")
    for op in path:
        if op == 'REMOVE':
            print(str(current) +".", "".join("REMOVE:".ljust(12)), a[:i] + "[{}]".format(a[i]), a[i+1:])
            a = a[:i] + a[i+1:]
            print("".join("   now:".ljust(15)), a,"\n")
            current += 1
            continue
            
        elif op == 'INSERT':
            print(str(current) +".", "".join("INSERT:".ljust(12)), a[:j] + "[{}]".format(b[j]) + a[j:])
            a = a[:j] + b[j] + a[j:]
            print("".join("   now:".ljust(15)), a,"\n")
            current += 1
            
        elif op == 'REPLACE':
            print(str(current) +".", "".join("REPLACE:".ljust(12)), a[:i] + "[{x}->{y}]".format(x=a[i], y=b[j]) + a[i+1:])
            a = a[:j] + b[j] + a[j+1:]
            print("".join("   now:".ljust(15)), a,"\n")
            current += 1
            
        i += 1
        j += 1
        
    print()
    print()

## Zad 3.

In [7]:
a = "los"
b = "kloc"

c = "Łódź"
d = "Lodz"

e = "kwintesencja"
f = "quintessence"

g = "ATGAATCTTACCGCCTCG"
h = "ATGAGGCTCTGGCCCCTG"

words = [(a,b), (c,d), (e,f), (g,h)]

In [8]:
def test():
    for (x,y) in words:
        edit_table = get_edit_table(x,y)
        dist = edit_table[-1][-1]
        print('\033[1m' + "EDIT:",x,"->",y)
        print('\033[0m' + "distance:", dist)
        path = get_path(x,y,edit_table)[0]
        show_operations(x,y,path)

In [9]:
test()

EDIT: los -> kloc
distance: 2

-------- OPERATIONS ------------------
1. INSERT:      [k]los
   now:         klos 

2. REPLACE:     klo[s->c]
   now:         kloc 



EDIT: Łódź -> Lodz
distance: 3

-------- OPERATIONS ------------------
1. REPLACE:     [Ł->L]ódź
   now:         Lódź 

2. REPLACE:     L[ó->o]dź
   now:         Lodź 

3. REPLACE:     Lod[ź->z]
   now:         Lodz 



EDIT: kwintesencja -> quintessence
distance: 5

-------- OPERATIONS ------------------
1. REPLACE:     [k->q]wintesencja
   now:         qwintesencja 

2. REPLACE:     q[w->u]intesencja
   now:         quintesencja 

3. INSERT:      quintes[s]encja
   now:         quintessencja 

4. REPLACE:     quintessenc[j->e]a
   now:         quintessencea 

5. REMOVE:      quintessence[a] 
   now:         quintessence 



EDIT: ATGAATCTTACCGCCTCG -> ATGAGGCTCTGGCCCCTG
distance: 7

-------- OPERATIONS ------------------
1. REPLACE:     ATGA[A->G]TCTTACCGCCTCG
   now:         ATGAGTCTTACCGCCTCG 

2. REPLACE:     ATGAG[T

## Zad 4. 

### Longest Common Subsequence

$LSC[i,j]$ $\rightarrow$ LCS for $x[1..i], y[1..j]$

$EDIT[i,j]$ $\rightarrow$ edit distance (insert/remove, without replacing)

$2 \cdot LCS[i,j] = i + j - EDIT[i,j]$

Returns only length of LCS (cannot find LCS)

In [10]:
def delta2(a,b):
    if a == b:
        return 0
    else:
        # replacing
        return 2

In [11]:
def LCS(a,b):
    return (len(a) + len(b) - edit_distance(a,b,delta2))/2

In [12]:
LCS("qwerty", "weronika")

3.0

## Zad 5.

In [13]:
import random

In [14]:
from spacy.tokenizer import Tokenizer
from spacy.lang.pl import Polish
nlp = Polish()
tokenizer = Tokenizer(nlp.vocab)

In [15]:
with open('romeo-i-julia-700.txt', "r", encoding="utf8") as file:
    text = file.read()

In [16]:
tokens = tokenizer(text)

## Zad 6.

In [17]:
tokens1 = [token for token in tokens if random.random() > 0.03]
tokens2 = [token for token in tokens if random.random() > 0.03]

In [18]:
len(tokens)

2272

In [19]:
len(tokens1)

2192

In [20]:
len(tokens2)

2211

In [21]:
with open('text1.txt', 'w', encoding="utf8") as out_file:
    for token in tokens1:
        out_file.write(token.text_with_ws)

In [22]:
with open('text2.txt', 'w', encoding="utf8") as out_file:
    for token in tokens2:
        out_file.write(token.text_with_ws)

## Zad 7.

#### Tokens LCS

In [23]:
LCS(tokens1,tokens2)

2134.0

#### Chars LCS

In [24]:
with open('text1.txt', "r", encoding="utf8") as file:
    text1 = file.read()

In [25]:
with open('text2.txt', "r", encoding="utf8") as file:
    text2 = file.read()

In [26]:
len(text1)

12139

In [27]:
len(text2)

12289

In [28]:
LCS(text1,text2)

11808.0

## Zad 8.

This version of LCS allows you to find not only the length but also longest subsequence present in both of strings.

In [29]:
from bisect import bisect

In [30]:
def LCS2(x, y):
    ranges = []
    ranges.append(len(y))
    y_letters = list(y)
    for i in range(len(x)):
        positions = [j for j, l in enumerate(y_letters) if l == x[i]]
        positions.reverse()
        for p in positions:
            k = bisect(ranges, p)
            if k == bisect(ranges, p - 1):
                if k < len(ranges) - 1:
                    ranges[k] = p
                else:
                    ranges.insert(k, p)
    return len(ranges) - 1

In [31]:
LCS2(tokens1, tokens2)

2134

### [diff](https://en.wikipedia.org/wiki/Diff)

lines was added(a) or deleted(d)

In [34]:
def diff(file1, file2):
    with open(file1, 'r', encoding="utf8") as file:
        text1 = file.read()
    
    with open(file2, 'r', encoding="utf8") as file:
        text2 = file.read()

        
    text1 = text1.split('\n')
    text2 = text2.split('\n')
    

    edit_table = get_edit_table(text1, text2, delta2)
    path, pos = get_path(text1, text2, edit_table)
      
    x = 0
        
    for j in range(len(path)):
        op = path[j]
        i = pos[j]
        if op == 'INSERT':
            print("> ({})  ".format(i + 1) + text1[i])
            x -= 1
        elif op == 'REMOVE':
            print("< ({})  ".format(i + x + 1) + text2[i + x])
            x += 1


## Zad 9.

In [35]:
diff('text1.txt', 'text2.txt')

> (4)  tłum. Józef Paszkowski
< (4)  Józef Paszkowski
> (13)   * MONTEKI, KAPULET — naczelnicy domów sobie
> (14)   * STARZEC — brat 
> (15)   ROMEO — syn Montekiego
> (16)   * MERKUCJO — krewny księcia
< (13)   * MONTEKI, KAPULET — naczelnicy dwóch domów nieprzyjaznych 
< (14)   * STARZEC — stryjeczny brat Kapuleta
< (15)   * ROMEO — syn Montekiego
< (16)   * MERKUCJO — krewny 
> (18)   * TYBALT — krewny Pani Kapulet* LAURENTY — ojciec franciszkanin
> (19)   * JAN — brat z tegoż zgromadzenia
< (18)   * TYBALT — krewny Pani Kapulet
> (32)   * Obywatele weroneńscy, różne osoby płci obojej, liczący się do przyjaciół domów, maski, straż wojskowa i inne osoby.
> (33)  
< (31)   * JULIA — córka Kapuletów
< (32)   * MARTA — mamka Julii
> (47)  Do nowej zbrodni pchają złości dawne,
> (48)  Plamiąc szlachetną krwią dłonie
< (46)  Dwa rody, zacne jednako i sławne —
< (47)  Tam, gdzie się rzecz ta rozgrywa, w Weronie,Do nowej zbrodni pchają złości dawne,
< (48)  Plamiąc szlachetną krwią szlachet